# Analyze network consequences of protein abundance changes - PCMT1
Currently set up for a single model
## Setup
### Import packages

In [ ]:
import io
import re
import shutil
import tempfile
import warnings
import zipfile
from operator import attrgetter
from pathlib import Path
from warnings import warn

import gurobipy as gp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sympy
from cobra.core import get_solution
from cobra.flux_analysis import find_blocked_reactions
from cobra.flux_analysis.variability import flux_variability_analysis
from cobra.manipulation import remove_genes
from rbc_gem_utils import (
    COBRA_CONFIGURATION,
    get_dirpath,
    handle_msg,
    read_cobra_model,
    show_versions,
    write_cobra_model,
)
from rbc_gem_utils.analysis.overlay import (
    DEFAULT_KEFF,
    DEFAULT_PREFIX_SUFFIX_VALUES,
    DEFAULT_PROTEOME_COMPARTMENT,
    ComplexDilution,
    ProteinDilution,
    add_relaxation_budget,
    load_overlay_model,
    update_slack_value,
)
from rbc_gem_utils.util import (
    AVOGADRO_NUMBER,
    DEFAULT_DRY_MASS_PER_CELL,
    DEFAULT_VOLUME_PER_CELL,
)

gp.setParam("OutputFlag", 0)
gp.setParam("LogToConsole", 0)

# Show versions of notebook
show_versions()

### Define configuration
#### COBRA Configuration

In [ ]:
COBRA_CONFIGURATION.solver = "gurobi"
# Set bound defaults much larger to prevent model loading issues due to protein constraint bounds
COBRA_CONFIGURATION.bounds = (-1e8, 1e8)
COBRA_CONFIGURATION

### Define organism, model, and dataset

In [ ]:
organism = "Human"
model_id = "RBC_GEM"
dataset_name = "DeepRed"

### Set computation options

In [ ]:
protein_of_interest = "PCMT1"
reaction_of_interest = "PROTISODMT_L"

ftype = "xml"  # In our experience, SBML/XML loads faster, but will take up to 4x more space uncompressed as compared to JSON
run_computations = True  # Keep off to use previously computed results
overwrite = True  # Whether to allow overwriting of previous simulation results
verbose = True

# Objective reactions
objective_reactions = ["NaKt"]
# Reactions that must have the capability to carry flux, sort for consistency
required_flux_reactions = []  # Add reactions to this list
required_flux_reactions = sorted(set(objective_reactions + required_flux_reactions))

min_relax_budget_for_objectives = True
# Remove blocked reactions before pcFVA simulation.
# For large models and/or multiple runs at different optimums, will speed up computation and potentially improve results.
remove_blocked_reactions = True
# Relaxation reactions that should be restricted to inactive
protein_relaxations_to_restrict = [protein_of_interest]
# Protein constraints that need lower bounds relaxed to prevent increase of
# associated subunits in direct conflict with physiologically known.
# Setting the lower bound prevents from being required.
protein_constraints_lb_to_relax = []

zip_kwargs = dict(compression=zipfile.ZIP_DEFLATED, compresslevel=None)

#### Set prefixes/suffixes to expect

In [ ]:
protein_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["proteins"]["prefix.dilution"]
protein_met_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["proteins"]["prefix.metabolite"]
relaxation_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["proteins"]["prefix.relaxation"]
enzyme_met_suffix_total = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["suffix.total"]
enzyme_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["prefix.dilution"]
enzyme_met_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["prefix.metabolite"]
enzyme_fwd_suffix = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["suffix.forward"]
enzyme_rev_suffix = DEFAULT_PREFIX_SUFFIX_VALUES["enzymes"]["suffix.reverse"]
budget_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["budgets"]["prefix.dilution"]
budget_met_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["budgets"]["prefix.metabolite"]
comp_suffix = f"_{DEFAULT_PROTEOME_COMPARTMENT}"

### Set figure options

In [ ]:
save_figures = True
transparent = False
imagetype = "svg"

### Set paths

In [ ]:
# Set paths
processed_data_dirpath = get_dirpath(use_temp="processed") / organism / dataset_name
# Set paths
overlay_dirpath = get_dirpath("analysis") / "OVERLAY" / organism
model_dirpath = overlay_dirpath / model_id
results_dirpath = (
    get_dirpath(use_temp="processed") / model_id / "OVERLAY" / organism / dataset_name
)
fitting_dirpath = results_dirpath / "fitting"
sample_pcmodels_dirpath = results_dirpath / "pcmodels"
abundance_change_results_path = results_dirpath / "abundance_changes"

# Ensure directories exist
abundance_change_results_path.mkdir(exist_ok=True, parents=True)

## Load RBC-GEM model

In [ ]:
model = read_cobra_model(filename=model_dirpath / f"{model_id}.xml")

ftype = "xml"
with zipfile.ZipFile(f"{sample_pcmodels_dirpath}.zip", "r") as zfile:
    with zfile.open(f"{model_id}_PC_{dataset_name}.{ftype}", "r") as model_file:
        pcmodel = load_overlay_model(
            filename=io.StringIO(model_file.read().decode("utf-8")), filetype=ftype
        )
pcmodel

### Set protein of interest

In [ ]:
protein_gene = pcmodel.genes.get_by_id(protein_of_interest)
protein_met = pcmodel.metabolites.get_by_id(
    f"{protein_met_prefix}{protein_of_interest}{comp_suffix}"
)
protein_rxn = pcmodel.reactions.get_by_id(f"{protein_rxn_prefix}{protein_met.id}")

# Store IDs for later use
protein_mid = protein_met.id
protein_rid = protein_rxn.id
protein_relax_id = f"{relaxation_rxn_prefix}{protein_mid}"

biochemical_reactions = []
for reaction in sorted(protein_gene.reactions, key=attrgetter("id")):
    if reaction in model.reactions:
        print(model.reactions.get_by_id(reaction.id))
        biochemical_reactions += [reaction.id]
required_flux_reactions = list(
    set(required_flux_reactions).union(biochemical_reactions)
)
protein_gene

### Get original values from model

In [ ]:
budget_met_relaxation = pcmodel.metabolites.get_by_id(f"{budget_met_prefix}relaxation")
budget_rxn_relaxation = pcmodel.reactions.get_by_id(
    f"{budget_rxn_prefix}{budget_met_prefix}relaxation"
)

orig_relax_budget = budget_rxn_relaxation.bounds
orig_protein_bounds = protein_rxn.bounds

### Define helper functions

In [ ]:
def find_and_remove_blocked_reactions(
    pcmodel, relaxation_rxns_required=None, prevent_removal=None, verbose=False
):
    pcmodel = pcmodel.copy()
    n_reactions_original = len(pcmodel.reactions)
    n_genes_original = len(pcmodel.genes)
    pcmodel.objective = sum(
        [
            r.flux_expression
            for r in pcmodel.reactions.get_by_any(required_flux_reactions)
        ]
    )
    if prevent_removal is None:
        prevent_removal = []
    for relax_rxn in pcmodel.reactions.query(
        lambda x: x.id.startswith(relaxation_rxn_prefix)
    ):
        if relax_rxn.id in relaxation_rxns_required:
            continue
        else:
            relax_rxn.bounds = (0, 0)
    reactions_to_remove = find_blocked_reactions(
        model=pcmodel,
        reaction_list=None,
        zero_cutoff=COBRA_CONFIGURATION.tolerance,
        open_exchanges=True,
        processes=min(60, COBRA_CONFIGURATION.processes),
    )
    reactions_to_remove = sorted(
        set(reactions_to_remove).difference(
            [
                rxn
                for prot in prevent_removal
                for rxn in [
                    f"{protein_rxn_prefix}{protein_met_prefix}{prot}{comp_suffix}",
                    f"{relaxation_rxn_prefix}{protein_met_prefix}{prot}{comp_suffix}",
                ]
            ]
        )
    )
    pcmodel.remove_reactions(reactions_to_remove, remove_orphans=True)
    genes_to_remove = [
        gene.id
        for gene in pcmodel.genes
        if not pcmodel.reactions.has_id(
            f"{protein_rxn_prefix}{protein_met_prefix}{gene.id}{comp_suffix}"
        )
        and not pcmodel.reactions.has_id(
            f"{relaxation_rxn_prefix}{protein_met_prefix}{gene.id}{comp_suffix}"
        )
    ]
    genes_to_remove = sorted(set(genes_to_remove).difference(prevent_removal))
    remove_genes(pcmodel, gene_list=genes_to_remove, remove_reactions=True)
    handle_msg(
        f"Number of blocked reactions removed: {n_reactions_original - len(pcmodel.reactions)}",
        print_msg=verbose,
    )
    handle_msg(
        f"Number of associated genes removed: {n_genes_original - len(pcmodel.genes)}",
        print_msg=verbose,
    )
    return pcmodel

### Ensure model can optimize all reactions catalyzed by protein of interest

In [ ]:
print("Old relaxation bounds: ({:.6f}, {:.6f})".format(*orig_relax_budget))

# Restrict relaxation reactions for specific proteins
for protein in protein_relaxations_to_restrict:
    protein_met = pcmodel.metabolites.get_by_id(
        f"{protein_met_prefix}{protein}{comp_suffix}"
    )
    relax_prot_rxn = pcmodel.reactions.get_by_id(
        f"{relaxation_rxn_prefix}{protein_met.id}"
    )
    relax_prot_rxn.bounds = (0, 0)
# Relax lower bound constraint for specific proteins
for protein in protein_constraints_lb_to_relax:
    protein_met = pcmodel.metabolites.get_by_id(
        f"{protein_met_prefix}{protein}{comp_suffix}"
    )
    protein_rxn = pcmodel.reactions.get_by_id(f"{protein_rxn_prefix}{protein_met.id}")
    protein_rxn.lower_bound = 0

budget_rxn_relaxation = pcmodel.reactions.get_by_id(
    f"{budget_rxn_prefix}{budget_met_prefix}relaxation"
)
# Determine smallest allowable relxation budget that allows flux through objectives and set as upper bound
with pcmodel:
    pcmodel.objective = (
        sum(
            [
                r.flux_expression
                for r in pcmodel.reactions.get_by_any(required_flux_reactions)
            ]
        )
        - budget_rxn_relaxation.flux_expression
    )
    pcmodel.objective_direction = "max"
    # Fail loudly, should not occur unless a restricted relaxation proteins are absolutely necessary
    pcmodel.slim_optimize(error_value=None)
    relaxation_rxns_required = get_solution(
        pcmodel,
        reactions=pcmodel.reactions.query(
            lambda x: x.id.startswith(relaxation_rxn_prefix)
        ),
    )
    relaxation_rxns_required = set(
        relaxation_rxns_required.fluxes[
            relaxation_rxns_required.fluxes != 0
        ].index.to_list()
    )
    budget_min = budget_rxn_relaxation.flux
if min_relax_budget_for_objectives:
    budget_rxn_relaxation.upper_bound = budget_min
print("New relaxation bounds: ({:.6f}, {:.6f})".format(*budget_rxn_relaxation.bounds))
with warnings.catch_warnings(action="ignore"):
    pcmodel = find_and_remove_blocked_reactions(
        pcmodel,
        relaxation_rxns_required=relaxation_rxns_required,
        prevent_removal=list(
            set(protein_relaxations_to_restrict)
            .union(protein_constraints_lb_to_relax)
            .union([protein_of_interest])
        ),
        verbose=verbose,
    )

pcmodel.objective = sum(
    [r.flux_expression for r in pcmodel.reactions.get_by_any(required_flux_reactions)]
)

sol = pcmodel.optimize(raise_error=True)

# Display solution for required flux capable reactions
objective_sol = sol.fluxes.loc[required_flux_reactions]
print(f"Optimization objective: {sol.objective_value:.6f}\n{objective_sol}")
# Protein utilized
protein_sol = sol.fluxes.loc[[protein_rid, protein_relax_id]]
print(f"\nProtein utilized: {protein_sol.sum():.6f}\n{protein_sol}")
# Required relaxation budget
relax_sol = sol.fluxes.loc[
    pcmodel.reactions.query(
        lambda x: x.id.startswith(relaxation_rxn_prefix)
        or x.id.endswith(budget_met_relaxation.id)
    ).list_attr("id")
]
relax_sol = relax_sol[relax_sol != 0]
relax_budget_used = relax_sol.loc[budget_rxn_relaxation.id]
print(
    f"\nTotal relaxation budget utilized: {relax_budget_used:.6f} ({relax_budget_used / budget_rxn_relaxation.upper_bound:.4%})\n{relax_sol}"
)
pcmodel

## Optimal fluxes as a function of reduction in protein abundance

In [ ]:
percent_interval = (0, -100)
percent_array = np.linspace(
    *percent_interval, 1 + (percent_interval[0] - percent_interval[1])
)

proteome_rxns_of_interest = [
    protein_rid,
    protein_relax_id,
]
biochemical_rxns_of_interest = [
    reaction_of_interest,
]

solutions = []
with pcmodel:
    protein_reaction = pcmodel.reactions.get_by_id(protein_rid)
    pcmodel.reactions.get_by_id("PROTDMT_D").bounds = (0, 0)
    pcmodel.objective = sum(
        [
            rxn.flux_expression
            for rxn in pcmodel.reactions.get_by_any(required_flux_reactions)
        ]
    )

    for percent in percent_array:
        protein_reaction.bounds = (
            orig_protein_bounds[0] * (1 + percent / 100),
            orig_protein_bounds[1] * (1 + percent / 100),
        )
        sol = pcmodel.optimize()
        sols_of_interest = sol.fluxes.loc[
            biochemical_rxns_of_interest
            + [
                x
                for x in required_flux_reactions
                if x not in biochemical_rxns_of_interest
            ]
            + [x for x in proteome_rxns_of_interest if x not in required_flux_reactions]
        ]
        sols_of_interest.name = percent
        solutions.append(sols_of_interest)

df_solutions = pd.concat(solutions, axis=1)
df_solutions

### Plot results for flux as a function of protein abundance

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5.5, 4.5))
# sns.despine(fig)
approx_label_fmt = "$\\approx{:.5f}$"
xmin, xmax = tuple([-1 * x for x in percent_interval])
# Plot G6PDH2
flux_sol_of_interest = df_solutions.loc[[reaction_of_interest]].sum(axis=0)
flux_sol_of_interest_original = flux_sol_of_interest.copy()
# For reduction only, switch percentage sign for downward slope
flux_sol_of_interest.index *= -1
flux_sol_of_interest.plot(
    ax=ax,
    color="red",
    linewidth=3,
    label=f"Maximum flux through PIMT",
)
ax.annotate(
    "$\\text{SAM} + \\text{L-isoD} \\rightarrow \\text{SAH} + \\text{L-isoD}\ \\text{\\alpha -methyl ester}$\n",
    xy=(0.5, 1.00),
    ha="center",
    va="center",
    xycoords="axes fraction",
    fontsize="medium",
)


xpad = 0
ypad = 0.05
ax.set_xlim(xmin - xpad, xmax)
ax.set_ylim(0, flux_sol_of_interest.max() + ypad)
ax.xaxis.set_tick_params(labelsize="large")
ax.yaxis.set_tick_params(labelsize="large")
ax.set_xlabel(f"Reduction in PIMT abundance (%)", fontsize="x-large")
ax.set_ylabel("Flux (mmol/hr/gDW)", fontsize="xx-large")
ax.legend(
    bbox_to_anchor=(0.5, 1.3), loc="upper center", frameon=False, fontsize="xx-large"
)


ymax = flux_sol_of_interest.max()
ax.annotate(
    approx_label_fmt.format(ymax),
    xy=(100, ymax * 0.98),
    xycoords="data",
    fontsize="large",
)

pct = 84
ls = "--"
color = "black"
ax.vlines(
    x=pct,
    ymin=flux_sol_of_interest.min() - ypad,
    ymax=ymax,
    linestyle=ls,
    color=color,
    linewidth=1.5,
)
ax.hlines(y=ymax, xmin=pct, xmax=100, linestyle=ls, color=color, linewidth=1.5)

fig.tight_layout()
if save_figures:
    ftypes = ["png", "svg"]
    for ftype in ftypes:
        fig.savefig(
            abundance_change_results_path
            / f"Reduction_{protein_of_interest}_PanelA.{ftype}",
            transparent=transparent,
            format=ftype,
        )

## Modify rate constants
Assays performed using DSIP peptide with L-isoaspartyl-residue as substrate. Measured by SAH formation

Using $[E]_{T} = 500$ nmol Enzyme:

$$\begin{align}
\text{For wildtype:}&& K_\mathrm{m} = 11341 \ nM,\ && V_{max} = 428.3\ nmol\ product / min\ \\
\text{   For V120I:}&& K_\mathrm{m} = 12541 \ nM,\ && V_{max} = 591.8\ nmol\ product / min\ \\
\end{align}$$

In [ ]:
kinetic_parameters = {
    # 1:1 stoichiometry, nmol substrate = nmol product
    "WT": {
        "Vmax": 428.3 * 60,  # nmol product / min * 60 min / 1hr --> nmol product / hr
        "E_total": 500,  # nmol enzyme
        "Km": 11341,  # nmol substrate / L
    },
    "V120I": {
        "Vmax": 591.8 * 60,  # nmol product / min * 60 min / 1hr --> nmol product / hr
        "E_total": 500,  # nmol enzyme
        "Km": 12541,  # nmol substrate / L
    },
}
for key, value_dict in kinetic_parameters.items():
    kinetic_parameters[key].update(
        {
            "kcat": value_dict["Vmax"]
            / value_dict["E_total"]  # nmol product / (nmol enzyme * hr)
        }
    )

kinetic_parameters

#### Relative modification

In [ ]:
# models_dict = {}
# for key in list(kinetic_parameters):
#     pcmod = pcmodel.copy()
#     pcmod.id += f"_{key}"
#     reaction = pcmod.reactions.get_by_id(reaction_of_interest)
#     kcat_ratio = kinetic_parameters[key]["kcat"] / kinetic_parameters["WT"]["kcat"]
#     reaction.add_metabolites(
#         {
#             f"{enzyme_met_prefix}{reaction_of_interest}{direction}{comp_suffix}": sign * (1 / (DEFAULT_KEFF *  kcat_ratio) * 1e6)
#             for direction, sign in zip([enzyme_fwd_suffix, enzyme_rev_suffix], [-1, 1])
#         },
#         combine=False
#     )
#     print(f"For {key}:")
#     print(f"kcat ratio: {kcat_ratio}")
#     print(reaction)
#     print()
#     models_dict[key] = pcmod
# models_dict

In [ ]:
# percent_interval = (0, -100)
# percent_array =  np.linspace(*percent_interval, 1001)
# proteome_rxns_of_interest = [
#     protein_rid,
#     protein_relax_id,
# ]
# biochemical_rxns_of_interest = [
#     reaction_of_interest,
# ]
# solutions_dict = {}

# for key, pcmod in models_dict.items():
#     solutions = []
#     with pcmod:
#         protein_reaction = pcmod.reactions.get_by_id(protein_rid)
#         pcmod.objective = sum(
#             [
#                 rxn.flux_expression
#                 for rxn in pcmod.reactions.get_by_any(required_flux_reactions)
#             ]
#         )

#         for percent in percent_array:
#             protein_reaction.bounds = (
#                 orig_protein_bounds[0] * (1 + percent / 100),
#                 orig_protein_bounds[1] * (1 + percent / 100),
#             )
#             sol = pcmod.optimize()
#             sols_of_interest = sol.fluxes.loc[
#                 biochemical_rxns_of_interest
#                 + [
#                     x
#                     for x in required_flux_reactions
#                     if x not in biochemical_rxns_of_interest
#                 ]
#                 + [x for x in proteome_rxns_of_interest if x not in required_flux_reactions]
#             ]
#             sols_of_interest.name = percent
#             solutions.append(sols_of_interest)
#     solutions_dict[key] = pd.concat(solutions, axis=1)
# solutions_dict

In [ ]:
# colors = {"WT": "black", "V120I": "red"}
# linestyles = {"WT": "--", "V120I": "-"}


# fig, ax = plt.subplots(1, 1, figsize=(5.5, 4.5))

# xmin, xmax = tuple([-1 * x for x in percent_interval])
# ymin, ymax = (np.inf, -np.inf)
# for idx, (key, df_solutions) in enumerate(solutions_dict.items()):
#     flux_sol_of_interest = df_solutions.loc[[reaction_of_interest]].sum(axis=0)
#     flux_sol_of_interest.index *= -1
#     flux_sol_of_interest.plot(
#         ax=ax,
#         color=colors[key],
#         linestyle=linestyles[key],
#         linewidth=2,
#         label=key,
#         zorder=idx+3 - 2*idx,
#     )
#     ymin = min(flux_sol_of_interest.min(), ymin)
#     ymax = max(flux_sol_of_interest.max(), ymax)
#     pct = flux_sol_of_interest[flux_sol_of_interest.diff() != 0].index[1]
#     print(pct)
#     ax.vlines(
#         x=pct,
#         ymin=flux_sol_of_interest.min() - ypad,
#         ymax=flux_sol_of_interest.max(),
#         linestyle=":",
#         color=colors[key],
#         linewidth=1,
#     )


# ax.annotate(
#     "$\\text{SAM} + \\text{L-isoD} \\rightarrow \\text{SAH} + \\text{L-isoD}\ \\alpha\\text{-methyl ester}$\n",
#     xy=(0.5, 1.00),
#     ha="center",
#     va="center",
#     xycoords="axes fraction",
#     fontsize="medium",
# )


# xpad = 0
# ypad = 0.05
# ax.set_xlim(xmin - xpad, xmax + xpad)
# ax.set_ylim(ymin, ymax + ypad)
# ax.xaxis.set_tick_params(labelsize="large")
# ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(20, offset=0))
# ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(5, offset=0))
# ax.yaxis.set_tick_params(labelsize="large")
# ax.set_xlabel(f"Reduction in PIMT abundance (%)", fontsize="x-large")
# ax.set_ylabel("Flux (mmol/hr/gDW)", fontsize="xx-large")
# ax.legend(
#     bbox_to_anchor=(0.5, 1.3), loc="upper center", frameon=False, fontsize="xx-large", ncol=2,
# )


# ax.annotate(
#     "$\\approx${:.4f}".format(ymax),
#     xy=(100, ymax * 0.98),
#     xycoords="data",
#     fontsize="large",
# )


# fig.tight_layout()
# if save_figures:
#     ftypes = ["png", "svg"]
#     for ftype in ftypes:
#         fig.savefig(
#             abundance_change_results_path
#             / f"Reduction_{protein_of_interest}_PanelA.{ftype}",
#             transparent=transparent,
#             format=ftype,
#         )

#### Direct integration

In [ ]:
models_dict = {}
for key in list(kinetic_parameters):
    pcmod = pcmodel.copy()
    pcmod.id += f"_{key}"
    reaction = pcmod.reactions.get_by_id(reaction_of_interest)
    reaction.add_metabolites(
        {
            f"{enzyme_met_prefix}{reaction_of_interest}{direction}{comp_suffix}": sign
            * (1 / kinetic_parameters[key]["kcat"])
            * 1e6
            for direction, sign in zip([enzyme_fwd_suffix, enzyme_rev_suffix], [-1, 1])
        },
        combine=False,
    )
    print(f"For {key}:")
    print(reaction)
    print()
    models_dict[key] = pcmod
models_dict

In [ ]:
percent_interval = (0, -100)
percent_array = np.linspace(*percent_interval, 1001)
proteome_rxns_of_interest = [
    protein_rid,
    protein_relax_id,
]
biochemical_rxns_of_interest = [
    reaction_of_interest,
]
solutions_dict = {}

for key, pcmod in models_dict.items():
    solutions = []
    with pcmod:
        protein_reaction = pcmod.reactions.get_by_id(protein_rid)
        pcmod.reactions.get_by_id("PROTDMT_D").bounds = (0, 0)
        pcmod.objective = sum(
            [
                rxn.flux_expression
                for rxn in pcmod.reactions.get_by_any(required_flux_reactions)
            ]
        )

        for percent in percent_array:
            protein_reaction.bounds = (
                orig_protein_bounds[0] * (1 + percent / 100),
                orig_protein_bounds[1] * (1 + percent / 100),
            )
            sol = pcmod.optimize()
            sols_of_interest = sol.fluxes.loc[
                biochemical_rxns_of_interest
                + [
                    x
                    for x in required_flux_reactions
                    if x not in biochemical_rxns_of_interest
                ]
                + [
                    x
                    for x in proteome_rxns_of_interest
                    if x not in required_flux_reactions
                ]
            ]
            sols_of_interest.name = percent
            solutions.append(sols_of_interest)
    solutions_dict[key] = pd.concat(solutions, axis=1)
solutions_dict;

In [ ]:
colors = {"WT": "black", "V120I": "#F19A95"}
linestyles = {"WT": "-", "V120I": "-"}


fig, ax = plt.subplots(1, 1, figsize=(4.5, 4.5))

xmin, xmax = tuple([-1 * x for x in percent_interval])
ymin, ymax = (np.inf, -np.inf)
xpad_percent = 0.0
ypad_percent = 0.1
annotation_ls = "--"

for idx, (key, df_solutions) in enumerate(solutions_dict.items()):
    flux_sol_of_interest = df_solutions.loc[[reaction_of_interest]].sum(axis=0)
    flux_sol_of_interest.index *= -1
    flux_sol_of_interest *= 1e6
    flux_sol_of_interest.plot(
        ax=ax,
        color=colors[key],
        linestyle=linestyles[key],
        linewidth=2,
        label=key,
        zorder=4 - idx,
    )
    ymin = min(flux_sol_of_interest.min(), ymin)
    ymax = max(flux_sol_of_interest.max(), ymax)
    pct = 0
    ax.hlines(
        y=flux_sol_of_interest.loc[pct],
        xmin=pct,
        xmax=xmax + abs(xmax * xpad_percent),
        linestyle=annotation_ls,
        color=colors[key],
        linewidth=1,
        zorder=idx + 3 - 2 * idx,
    )
    ax.annotate(
        "$\\approx${:.1f}".format(ymax),
        xy=(xmax + abs(xmax * xpad_percent), ymax),
        xycoords="data",
        fontsize="large",
        va="bottom",
        ha="right",
        color=colors[key],
    )


# ax.annotate(
#     "$\\text{SAM} + \\text{L-isoD} \\rightarrow \\text{SAH} + \\text{L-isoD}\ \\alpha\\text{-methyl ester}$\n",
#     xy=(0.5, 1.00),
#     ha="center",
#     va="center",
#     xycoords="axes fraction",
#     fontsize="medium",
# )


ax.set_xlim(xmin - abs(xmin * xpad_percent), xmax + abs(xmax * xpad_percent))
ax.set_ylim(ymin - abs(ymin * ypad_percent), ymax + abs(ymax * ypad_percent))
ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(20, offset=0))
ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(5, offset=0))
# ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.0002, offset=0))
# ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.0001, offset=0))
ax.xaxis.set_tick_params(labelsize="large")
ax.yaxis.set_tick_params(labelsize="large")
ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
ax.set_xlabel(f"Reduction in PIMT abundance (%)", fontsize="x-large")
ax.set_ylabel("Flux (nmol/hr/gDW)", fontsize="xx-large")
ax.legend(
    bbox_to_anchor=(0.5, 1.2),
    loc="upper center",
    frameon=False,
    fontsize="xx-large",
    ncol=2,
)

# Determime what percent reduction makes V120I activity equal WT
wt_max = solutions_dict["WT"].loc[reaction_of_interest].max()
df = solutions_dict["V120I"].loc[reaction_of_interest]
df.index *= -1
# Swap index to interpolate percentage
df = df.reset_index(drop=False).set_index(reaction_of_interest).copy()
df.loc[wt_max] = np.nan
df = df.sort_index().interpolate()
pct = df.loc[wt_max].item()
ax.plot(pct, wt_max * 1e6, marker="o", ls="None", color="black", zorder=5)
# ax.vlines(
#     x=pct,
#     ymin=flux_sol_of_interest.min() - abs(flux_sol_of_interest.min() * ypad),
#     ymax=wt_max,
#     linestyle=":",
#     color="grey",
#     linewidth=1,
# )
# ax.annotate(
#     "$\\approx${:.2%}".format(pct/100),
#     xy=(pct, wt_max),
#     xycoords="data",
#     fontsize="large",
#     va="bottom",
#     ha="left",
# )

fig.tight_layout()
if save_figures:
    ftypes = ["png", "svg"]
    for ftype in ftypes:
        fig.savefig(
            abundance_change_results_path
            / f"Reduction_{protein_of_interest}_percent.{ftype}",
            transparent=transparent,
            format=ftype,
        )

In [ ]:
colors = {"WT": "black", "V120I": "#F19A95"}
linestyles = {"WT": "-", "V120I": "-"}


fig, ax = plt.subplots(1, 1, figsize=(4.5, 4.5))

ymin, ymax = (np.inf, -np.inf)
xpad_percent = 0.0
ypad_percent = 0.1
annotation_ls = "--"

for idx, (key, df_solutions) in enumerate(solutions_dict.items()):
    df_solutions = df_solutions.T.set_index(protein_reaction.id)
    xmin, xmax = (df_solutions.index.min(), df_solutions.index.max())
    flux_sol_of_interest = df_solutions.T.loc[[reaction_of_interest]].sum(axis=0)
    flux_sol_of_interest *= 1e6
    # flux_sol_of_interest.index *= -1
    flux_sol_of_interest.plot(
        ax=ax,
        color=colors[key],
        linestyle=linestyles[key],
        linewidth=2,
        label=key,
        zorder=4 - idx,
    )
    ymin = min(flux_sol_of_interest.min(), ymin)
    ymax = max(flux_sol_of_interest.max(), ymax)
    ax.hlines(
        y=flux_sol_of_interest.loc[flux_sol_of_interest.index.max()],
        xmin=flux_sol_of_interest.index.min(),
        xmax=xmax + abs(xmax * xpad_percent),
        linestyle=annotation_ls,
        color=colors[key],
        linewidth=1,
        zorder=idx + 3 - 2 * idx,
    )
    ax.annotate(
        "$\\approx${:.0f}".format(ymax),
        xy=(xmax + abs(xmax * xpad_percent), ymax * 0.97),
        xycoords="data",
        fontsize="large",
        va="bottom",
        ha="left",
        color=colors[key],
    )


# ax.annotate(
#     "$\\text{SAM} + \\text{L-isoD} \\rightarrow \\text{SAH} + \\text{L-isoD}\ \\alpha\\text{-methyl ester}$\n",
#     xy=(0.5, 1.00),
#     ha="center",
#     va="center",
#     xycoords="axes fraction",
#     fontsize="medium",
# )


ax.set_xlim(xmin - abs(xmin * xpad_percent), xmax + abs(xmax * xpad_percent))
ax.set_ylim(ymin - abs(ymin * ypad_percent), ymax + abs(ymax * ypad_percent))
ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(7))
# ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(5, offset=0))
# ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.0002, offset=0))
# ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.0001, offset=0))
ax.xaxis.set_tick_params(labelsize="large")
ax.yaxis.set_tick_params(labelsize="large")
ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
ax.set_xlabel(f"Active PIMT (nmol/gDW)", fontsize="x-large")
ax.set_ylabel("Flux (nmol/hr/gDW)", fontsize="xx-large")
ax.legend(
    bbox_to_anchor=(0.5, 1.2),
    loc="upper center",
    frameon=False,
    fontsize="xx-large",
    ncol=2,
)

# Determime what abundance value makes V120I activity equal WT
wt_max = solutions_dict["WT"].loc[reaction_of_interest].max()
df = solutions_dict["V120I"].loc[[protein_reaction.id, reaction_of_interest]].T
# # Swap index to interpolate percentage
df = df.set_index(reaction_of_interest).copy()
df.loc[wt_max] = np.nan
df = df.sort_index().interpolate()
ax.plot(
    df.loc[wt_max].item(), wt_max * 1e6, marker="o", ls="None", color="black", zorder=5
)
# ax.vlines(
#     x=df.loc[wt_max].item(),
#     ymin=flux_sol_of_interest.min() - abs(flux_sol_of_interest.min() * ypad),
#     ymax=wt_max,
#     linestyle=":",
#     color="grey",
#     linewidth=1,
# )
# ax.annotate(
#     "$\\approx${:.2%}".format(pct/100),
#     xy=(pct, wt_max),
#     xycoords="data",
#     fontsize="large",
#     va="bottom",
#     ha="left",
# )

fig.tight_layout()
if save_figures:
    ftypes = ["png", "svg"]
    for ftype in ftypes:
        fig.savefig(
            abundance_change_results_path
            / f"Reduction_{protein_of_interest}_absolute.{ftype}",
            transparent=transparent,
            format=ftype,
        )